In [1]:
##The feature engineering was redone starting from the original work and with extra work by us.

In [1]:

#updates:
# added magic feature - boost the AUC by 0.0003

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from scipy import stats
from plotly.offline import iplot, init_notebook_mode

init_notebook_mode(connected=True)

# Preprocessing, modelling and evaluating
from sklearn import preprocessing
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold, cross_val_score, KFold

import gc
import os 

#print where those datasets are located 
from sklearn.preprocessing import minmax_scale
# from sklearn.cluster import KMeans

## Function to reduce the DF size
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df


#df_trans = pd.read_csv('train_transaction.csv')
df_trans = pd.read_csv('train_transaction.csv',index_col='TransactionID')
df_test_trans = pd.read_csv('test_transaction.csv',index_col='TransactionID')

#record index of test datasets
test_trans_id = df_test_trans.index

#df_id = pd.read_csv('train_identity.csv')
df_id = pd.read_csv('train_identity.csv',index_col='TransactionID')
df_test_id = pd.read_csv('test_identity.csv',index_col='TransactionID')




sample_submission = pd.read_csv('sample_submission.csv', index_col='TransactionID')


In [2]:
def id_split(dataframe):
    
    #expand = split and expand to seperate column
    dataframe['device_name'] = dataframe['DeviceInfo'].str.split('/', expand=True)[0]
    dataframe['device_version'] = dataframe['DeviceInfo'].str.split('/', expand=True)[1]

    dataframe['OS_id_30'] = dataframe['id_30'].str.split(' ', expand=True)[0]
    dataframe['version_id_30'] = dataframe['id_30'].str.split(' ', expand=True)[1]

    dataframe['browser_id_31'] = dataframe['id_31'].str.split(' ', expand=True)[0]
    dataframe['version_id_31'] = dataframe['id_31'].str.split(' ', expand=True)[1]

    dataframe['screen_width'] = dataframe['id_33'].str.split('x', expand=True)[0]
    dataframe['screen_height'] = dataframe['id_33'].str.split('x', expand=True)[1]

    dataframe['id_34'] = dataframe['id_34'].str.split(':', expand=True)[1]
    dataframe['id_23'] = dataframe['id_23'].str.split(':', expand=True)[1]
    
    #very specifically group device brands to help making decision
    dataframe.loc[dataframe['device_name'].str.contains('SM', na=False), 'device_name'] = 'Samsung'
    dataframe.loc[dataframe['device_name'].str.contains('SAMSUNG', na=False), 'device_name'] = 'Samsung'
    dataframe.loc[dataframe['device_name'].str.contains('GT-', na=False), 'device_name'] = 'Samsung'
    dataframe.loc[dataframe['device_name'].str.contains('Moto G', na=False), 'device_name'] = 'Motorola'
    dataframe.loc[dataframe['device_name'].str.contains('Moto', na=False), 'device_name'] = 'Motorola'
    dataframe.loc[dataframe['device_name'].str.contains('moto', na=False), 'device_name'] = 'Motorola'
    dataframe.loc[dataframe['device_name'].str.contains('LG-', na=False), 'device_name'] = 'LG'
    dataframe.loc[dataframe['device_name'].str.contains('rv:', na=False), 'device_name'] = 'RV'
    dataframe.loc[dataframe['device_name'].str.contains('HUAWEI', na=False), 'device_name'] = 'Huawei'
    dataframe.loc[dataframe['device_name'].str.contains('ALE-', na=False), 'device_name'] = 'Huawei'
    dataframe.loc[dataframe['device_name'].str.contains('-L', na=False), 'device_name'] = 'Huawei'
    dataframe.loc[dataframe['device_name'].str.contains('Blade', na=False), 'device_name'] = 'ZTE'
    dataframe.loc[dataframe['device_name'].str.contains('BLADE', na=False), 'device_name'] = 'ZTE'
    dataframe.loc[dataframe['device_name'].str.contains('Linux', na=False), 'device_name'] = 'Linux'
    dataframe.loc[dataframe['device_name'].str.contains('XT', na=False), 'device_name'] = 'Sony'
    dataframe.loc[dataframe['device_name'].str.contains('HTC', na=False), 'device_name'] = 'HTC'
    dataframe.loc[dataframe['device_name'].str.contains('ASUS', na=False), 'device_name'] = 'Asus'

    #make the device name that are less than 200 to be other
    dataframe.loc[dataframe.device_name.isin(dataframe.device_name.value_counts()[dataframe.device_name.value_counts() < 200].index), 'device_name'] = "Others"
    
    #new column to record that this device has id???
    dataframe['had_id'] = 1
    gc.collect()
    
    return dataframe

In [4]:
df_trans[df_trans['V326']==0]['isFraud']

TransactionID
2987004    0
2987008    0
2987016    0
2987017    0
2987022    0
          ..
3577465    0
3577495    0
3577506    0
3577526    1
3577531    0
Name: isFraud, Length: 72668, dtype: int64

In [3]:

#C sum feature
df_trans["C_sum"] = df_trans.loc[:,'C1':'C14'].sum(axis = 1).astype(np.int8)
df_test_trans["C_sum"] = df_test_trans.loc[:,'C1':'C14'].sum(axis = 1).astype(np.int8)

    
##D1 - D15
df_trans['D_na'] =df_trans.loc[:,'D1':'D14'].isna().sum(axis=1).astype(np.int8)
df_test_trans['D_na'] =df_test_trans.loc[:,'D1':'D14'].isna().sum(axis=1).astype(np.int8)

    
##M1-9
i_cols = ['M1','M2','M3','M5','M6','M7','M8','M9']

df_trans['M_na'] = df_trans[i_cols].isna().sum(axis=1).astype(np.int8)
df_test_trans['M_na'] = df_test_trans[i_cols].isna().sum(axis=1).astype(np.int8)


    
##V_ features
df_trans['V_na'] =df_trans.loc[:,"V1":"V339"].isna().sum(axis=1).astype(np.int8)
df_test_trans['V_na'] =df_test_trans.loc[:,"V1":"V339"].isna().sum(axis=1).astype(np.int8)

##ids 
df_id['id_na'] = df_id.loc[:,"id_01":"id_38"].isna().sum(axis=1).astype(np.int8)
df_test_id['id_na'] = df_test_id.loc[:,"id_01":"id_38"].isna().sum(axis=1).astype(np.int8)




In [4]:
df_id = id_split(df_id)
df_test_id = id_split(df_test_id)

In [5]:

##do not need to include right_index
df_train = df_trans.merge(df_id, how='left', left_index=True,  on='TransactionID')
df_test = df_test_trans.merge(df_test_id, how='left', left_index=True, on='TransactionID')

#add total na for train and test data. 
df_train['tot_na'] = df_train.isna().sum(axis=1).astype(np.int8) 
df_test['tot_na'] = df_test.isna().sum(axis=1).astype(np.int8)

print(df_train.head())
print(df_test.head())


# y_train = df_train['isFraud'].copy()
del df_trans, df_id, df_test_trans, df_test_id
gc.collect()



df_train = reduce_mem_usage(df_train)
df_test = reduce_mem_usage(df_test)
##Deal with missing data



               isFraud  TransactionDT  TransactionAmt ProductCD  card1  card2  \
TransactionID                                                                   
2987000              0          86400            68.5         W  13926    NaN   
2987001              0          86401            29.0         W   2755  404.0   
2987002              0          86469            59.0         W   4663  490.0   
2987003              0          86499            50.0         W  18132  567.0   
2987004              0          86506            50.0         H   4497  514.0   

               card3       card4  card5   card6  ...  device_name  \
TransactionID                                    ...                
2987000        150.0    discover  142.0  credit  ...          NaN   
2987001        150.0  mastercard  102.0  credit  ...          NaN   
2987002        150.0        visa  166.0   debit  ...          NaN   
2987003        150.0  mastercard  117.0   debit  ...          NaN   
2987004        150

In [6]:
df_train.loc[3577531,"screen_height"]

'1600'

In [7]:
#https://www.kaggle.com/davidcairuz/feature-engineering-lightgbm#
#removed V300 feature
useful_features = ['TransactionDT','TransactionAmt', 'ProductCD', 'card1', 'card2', 'card3', 'card4', 'card5', 'card6', 'addr1', 'addr2', 'dist1',
                   'P_emaildomain', 'R_emaildomain', 'C1', 'C2', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11', 'C12', 'C13',
                   'C14', 'D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D8', 'D9', 'D10', 'D11', 'D12', 'D13', 'D14', 'D15', 'M2', 'M3',
                   'M4', 'M5', 'M6', 'M7', 'M8', 'M9', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 'V13', 'V17',
                   'V19', 'V20', 'V29', 'V30', 'V33', 'V34', 'V35', 'V36', 'V37', 'V38', 'V40', 'V44', 'V45', 'V46', 'V47', 'V48',
                   'V49', 'V51', 'V52', 'V53', 'V54', 'V56', 'V58', 'V59', 'V60', 'V61', 'V62', 'V63', 'V64', 'V69', 'V70', 'V71',
                   'V72', 'V73', 'V74', 'V75', 'V76', 'V78', 'V80', 'V81', 'V82', 'V83', 'V84', 'V85', 'V87', 'V90', 'V91', 'V92',
                   'V93', 'V94', 'V95', 'V96', 'V97', 'V99', 'V100', 'V126', 'V127', 'V128', 'V130', 'V131', 'V138', 'V139', 'V140',
                   'V143', 'V145', 'V146', 'V147', 'V149', 'V150', 'V151', 'V152', 'V154', 'V156', 'V158', 'V159', 'V160', 'V161',
                   'V162', 'V163', 'V164', 'V165', 'V166', 'V167', 'V169', 'V170', 'V171', 'V172', 'V173', 'V175', 'V176', 'V177',
                   'V178', 'V180', 'V182', 'V184', 'V187', 'V188', 'V189', 'V195', 'V197', 'V200', 'V201', 'V202', 'V203', 'V204',
                   'V205', 'V206', 'V207', 'V208', 'V209', 'V210', 'V212', 'V213', 'V214', 'V215', 'V216', 'V217', 'V219', 'V220',
                   'V221', 'V222', 'V223', 'V224', 'V225', 'V226', 'V227', 'V228', 'V229', 'V231', 'V233', 'V234', 'V238', 'V239',
                   'V242', 'V243', 'V244', 'V245', 'V246', 'V247', 'V249', 'V251', 'V253', 'V256', 'V257', 'V258', 'V259', 'V261',
                   'V262', 'V263', 'V264', 'V265', 'V266', 'V267', 'V268', 'V270', 'V271', 'V272', 'V273', 'V274', 'V275', 'V276',
                   'V277', 'V278', 'V279', 'V280', 'V282', 'V283', 'V285', 'V287', 'V288', 'V289', 'V291', 'V292', 'V294','id_01',
                   'id_02', 'id_03', 'id_05', 'id_06', 'id_09',
                   'id_11', 'id_12', 'id_13', 'id_14', 'id_15', 'id_17', 'id_19', 'id_20', 'id_30', 'id_31', 'id_32', 'id_33',
                   'id_36', 'id_37', 'id_38', 'DeviceType', 'DeviceInfo', 'device_name', 'device_version', 'OS_id_30', 'version_id_30',
                   'browser_id_31', 'version_id_31', 'screen_width', 'screen_height', 'had_id','tot_na','C_sum','D_na','M_na','V_na','id_na']
print(len(useful_features))

273


In [8]:
cols_to_drop = [col for col in df_train.columns if col not in useful_features]
cols_to_drop.remove('isFraud')


In [9]:
df_train = df_train.drop(cols_to_drop, axis=1)
df_test = df_test.drop(cols_to_drop, axis=1)

In [10]:
df_train

,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,...,device_name,device_version,OS_id_30,version_id_30,browser_id_31,version_id_31,screen_width,screen_height,had_id,tot_na
TransactionID,,,,,,,,,,,,,,,,,,,,,
2987000,0,86400,68.500000,W,13926,NaN,150.0,discover,142.0,credit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-12
2987001,0,86401,29.000000,W,2755,404.0,150.0,mastercard,102.0,credit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-16
2987002,0,86469,59.000000,W,4663,490.0,150.0,visa,166.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-35
2987003,0,86499,50.000000,W,18132,567.0,150.0,mastercard,117.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-19
2987004,0,86506,50.000000,H,4497,514.0,150.0,mastercard,102.0,credit,...,Samsung,NRD90M,Android,7.0,samsung,browser,2220,1080,1.0,-119
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3577535,0,15811047,49.000000,W,6550,NaN,150.0,visa,226.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-16
3577536,0,15811049,39.500000,W,10444,225.0,150.0,mastercard,224.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-34
3577537,0,15811079,30.953125,W,12037,595.0,150.0,mastercard,224.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-29


In [11]:
print(df_train)
print(df_test)

               isFraud  TransactionDT  TransactionAmt ProductCD  card1  card2  \
TransactionID                                                                   
2987000              0          86400       68.500000         W  13926    NaN   
2987001              0          86401       29.000000         W   2755  404.0   
2987002              0          86469       59.000000         W   4663  490.0   
2987003              0          86499       50.000000         W  18132  567.0   
2987004              0          86506       50.000000         H   4497  514.0   
...                ...            ...             ...       ...    ...    ...   
3577535              0       15811047       49.000000         W   6550    NaN   
3577536              0       15811049       39.500000         W  10444  225.0   
3577537              0       15811079       30.953125         W  12037  595.0   
3577538              0       15811088      117.000000         W   7826  481.0   
3577539              0      

In [12]:
#new feature regarding TransactionDT
# any repeated transation consecutively under one card1 number is very suspicious. 
df_train= df_train.sort_values(by =['card1','TransactionDT'])

rpt_trans = [1  if df_train.iloc[i,2] == df_train.iloc[i+1,2] or df_train.iloc[i,2] == df_train.iloc[i-1,2] else 0 for i in range(1,df_train.shape[0]-1)]
df_train['rept_trans'] =[0]+ rpt_trans +[0]

df_test= df_test.sort_values(by =['card1','TransactionDT'])

rpt_trans = [1  if df_test.iloc[i,2] == df_test.iloc[i+1,2] or df_test.iloc[i,2] == df_test.iloc[i-1,2] else 0 for i in range(1,df_test.shape[0]-1)]
df_test['rept_trans'] =[0]+ rpt_trans +[0]

In [13]:
#added card6 - debit credit
columns_a = ['TransactionAmt', 'id_02', 'D15']
columns_b = ['card1', 'card2']

for col_a in columns_a:
    for col_b in columns_b:
        for df in [df_train, df_test]:
            df[f'{col_a}_to_mean_{col_b}'] = df[col_a] / df.groupby([col_b])[col_a].transform('mean')
            df[f'{col_a}_to_std_{col_b}'] = df[col_a] / df.groupby([col_b])[col_a].transform('std')

In [14]:
# New feature - log of transaction amount.
df_train['TransactionAmt_Log'] = np.log(df_train['TransactionAmt'])
df_test['TransactionAmt_Log'] = np.log(df_test['TransactionAmt'])

# New feature - decimal part of the transaction amount.
df_train['TransactionAmt_decimal'] = ((df_train['TransactionAmt'] - df_train['TransactionAmt'].astype(int)) * 1000).astype(int)
df_test['TransactionAmt_decimal'] = ((df_test['TransactionAmt'] - df_test['TransactionAmt'].astype(int)) * 1000).astype(int)

# New feature - day of week in which a transaction happened.
df_train['Transaction_day_of_week'] = np.floor((df_train['TransactionDT'] / (3600 * 24) - 1) % 7)
df_test['Transaction_day_of_week'] = np.floor((df_test['TransactionDT'] / (3600 * 24) - 1) % 7)

# New feature - hour of the day in which a transaction happened.
df_train['Transaction_hour'] = np.floor(df_train['TransactionDT'] / 3600) % 24
df_test['Transaction_hour'] = np.floor(df_test['TransactionDT'] / 3600) % 24

# Some arbitrary features interaction
for feature in ['id_02__id_20', 'id_02__D8', 'D11__DeviceInfo', 'DeviceInfo__P_emaildomain',
                'card2__dist1', 'card1__card5', 'card2__id_20', 'card1__P_emaildomain', 'addr1__card1']:

    f1, f2 = feature.split('__')
    df_train[feature] = df_train[f1].astype(str) + '_' + df_train[f2].astype(str)
    df_test[feature] = df_test[f1].astype(str) + '_' + df_test[f2].astype(str)

    le = preprocessing.LabelEncoder()
    le.fit(list(df_train[feature].astype(str).values) + list(df_test[feature].astype(str).values))
    df_train[feature] = le.transform(list(df_train[feature].astype(str).values))
    df_test[feature] = le.transform(list(df_test[feature].astype(str).values))

# Encoding - count encoding for both df_train and df_test
for feature in ['card1', 'card2', 'card3', 'card4', 'card5', 'card6', 'id_36']:
    df_train[feature + '_count_full'] = df_train[feature].map(pd.concat([df_train[feature], df_test[feature]], ignore_index=True).value_counts(dropna=False))
    df_test[feature + '_count_full'] = df_test[feature].map(pd.concat([df_train[feature], df_test[feature]], ignore_index=True).value_counts(dropna=False))

# Encoding - count encoding separately for df_train and df_test
for feature in ['id_01', 'id_31', 'id_33', 'id_36']:
    df_train[feature + '_count_dist'] = df_train[feature].map(df_train[feature].value_counts(dropna=False))
    df_test[feature + '_count_dist'] = df_test[feature].map(df_test[feature].value_counts(dropna=False))

In [15]:
df_train = df_train.reset_index()
df_train

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,card2_count_full,card3_count_full,card4_count_full,card5_count_full,card6_count_full,id_36_count_full,id_01_count_dist,id_31_count_dist,id_33_count_dist,id_36_count_dist
0,3230924,0,5787419,23.4375,C,1000,555.0,185.0,mastercard,224.0,...,80404,109960,347386,153109,824959,267353,82170,5806,517251,134066
1,3023634,0,916268,183.0000,W,1001,555.0,150.0,visa,226.0,...,80404,956845,719649,553537,824959,819269,446307,450258,517251,449555
2,3151336,0,3504180,29.0000,W,1001,555.0,150.0,visa,226.0,...,80404,956845,719649,553537,824959,819269,446307,450258,517251,449555
3,3210739,0,5270458,27.0000,W,1001,555.0,150.0,visa,226.0,...,80404,956845,719649,553537,824959,819269,446307,450258,517251,449555
4,3020767,0,842821,150.0000,R,1004,583.0,150.0,visa,226.0,...,41503,956845,719649,553537,267648,267353,82170,22000,263,134066
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
590535,3419314,0,10938204,232.5000,W,18395,543.0,150.0,mastercard,224.0,...,9968,956845,347386,153109,824959,819269,446307,450258,517251,449555
590536,3420746,0,10964952,36.9375,W,18395,543.0,150.0,mastercard,224.0,...,9968,956845,347386,153109,824959,819269,446307,450258,517251,449555
590537,3446622,0,11806909,36.9375,W,18395,543.0,150.0,mastercard,224.0,...,9968,956845,347386,153109,824959,819269,446307,450258,517251,449555
590538,3480204,0,12865537,36.9375,W,18395,543.0,150.0,mastercard,224.0,...,9968,956845,347386,153109,824959,819269,446307,450258,517251,449555


In [16]:


#k fold cross validation target mean coding to avoid overfitting

N_SPLITS = 5
SEED = 6

folds = KFold(n_splits=N_SPLITS,shuffle = False, random_state=SEED)
# Test Data and expport DF
  
score = 0
y_result = 0

mean_of_target = df_train['isFraud'].mean()

target_mean_list =['card1','card2','addr1','addr1__card1','card1__card5','card1__P_emaildomain']

# kfold target mean encoding for regularisation

for fold_, (trn_idx, val_idx) in enumerate(folds.split(df_train)):
    print('Fold:',fold_+1)
    tr_x = df_train.iloc[trn_idx,:] 
    vl_x = df_train.iloc[val_idx,:]  
    
    for col in target_mean_list:
        print(col)
        df_train.loc[val_idx,f'{col}_target_enc'] = vl_x[col].map(tr_x.groupby(col)['isFraud'].mean())
        
        df_train[f'{col}_target_enc'].fillna(mean_of_target, inplace = True)
                                                        
for col in target_mean_list:
                                                            
    df_test[f'{col}_target_enc'] = df_test[col].map(df_train.groupby(col)[f'{col}_target_enc'].mean())

    df_test[f'{col}_target_enc'].fillna(mean_of_target, inplace = True)

    
    

Fold: 1
card1
card2
addr1
addr1__card1
card1__card5
card1__P_emaildomain
Fold: 2
card1
card2
addr1
addr1__card1
card1__card5
card1__P_emaildomain
Fold: 3
card1
card2
addr1
addr1__card1
card1__card5
card1__P_emaildomain
Fold: 4
card1
card2
addr1
addr1__card1
card1__card5
card1__P_emaildomain
Fold: 5
card1
card2
addr1
addr1__card1
card1__card5
card1__P_emaildomain


In [17]:
#label encoding the rest 
for col in df_train.columns:
    if df_train[col].dtype == 'object':
        le = preprocessing.LabelEncoder()
        le.fit(list(df_train[col].astype(str).values) + list(df_test[col].astype(str).values))
        df_train[col] = le.transform(list(df_train[col].astype(str).values))
        df_test[col] = le.transform(list(df_test[col].astype(str).values))

In [18]:
#why sort ??
X_train = df_train.sort_values('TransactionDT').drop(['isFraud', 'TransactionDT','TransactionID'], axis=1)
y_train = df_train.sort_values('TransactionDT')['isFraud']

X_test = df_test.drop(['TransactionDT'], axis=1)

del df_train, df_test
gc.collect()

20

In [19]:
X_test

,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,...,id_01_count_dist,id_31_count_dist,id_33_count_dist,id_36_count_dist,card1_target_enc,card2_target_enc,addr1_target_enc,addr1__card1_target_enc,card1__card5_target_enc,card1__P_emaildomain_target_enc
TransactionID,,,,,,,,,,,,,,,,,,,,,
4084529,50.0000,1,1001,555.0,150.0,4,226.0,2,269.0,87.0,...,82579,16054,6441,133287,0.03499,0.022705,0.035629,0.03499,0.03499,0.03499
4113026,39.0000,4,1001,555.0,150.0,4,226.0,2,269.0,87.0,...,364784,370066,436020,369714,0.03499,0.022705,0.035629,0.03499,0.03499,0.03499
4147003,60.0000,1,1002,NaN,150.0,2,NaN,1,343.0,87.0,...,82579,13098,2746,133287,0.03499,0.034990,0.017703,0.03499,0.03499,0.03499
3981622,42.9375,4,1003,452.0,150.0,2,224.0,2,264.0,87.0,...,364784,370066,436020,369714,0.03499,0.000650,0.018267,0.03499,0.03499,0.03499
4044940,100.0000,1,1004,583.0,150.0,4,226.0,1,325.0,87.0,...,12000,362,1555,3690,0.03499,0.041566,0.025488,0.03499,0.03499,0.03499
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4004795,117.0000,4,18396,111.0,150.0,4,226.0,2,126.0,87.0,...,364784,370066,436020,369714,0.03499,0.021892,0.025796,0.03499,0.03499,0.03499
4054400,226.0000,4,18396,111.0,150.0,4,226.0,2,126.0,87.0,...,364784,370066,436020,369714,0.03499,0.021892,0.025796,0.03499,0.03499,0.03499
4072739,226.0000,4,18396,111.0,150.0,4,226.0,2,126.0,87.0,...,364784,370066,436020,369714,0.03499,0.021892,0.025796,0.03499,0.03499,0.03499


In [20]:
X_train = X_train.fillna(-999)
X_test = X_test.fillna(-999)

##replace inf vlaues 



In [21]:
X_train[X_train> np.finfo(np.float32).max] = 50000
X_test[X_test> np.finfo(np.float32).max] = 50000

In [25]:
X_train.isnull().sum().sum()

0

In [24]:
np.where(X_train.values >= np.finfo(np.float32).max)

(array([], dtype=int64), array([], dtype=int64))

In [30]:
import lightgbm as lgb
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold, train_test_split
from sklearn import metrics
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier



from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression

models=[ 
            ######## First level ########
            [LGBMClassifier(
                    objective='binary',
                    n_estimators = 200,
                    boosting_type='gbdt',
                    metric='auc',
                    n_jobs=-1,
                    learning_rate=0.01,
                    num_leaves= 2**8,
                    max_depth=-1,
                    colsample_bytree= 0.7,
                    subsample_freq=1,
                    subsample=0.7,
                    max_bin=255,
                    verbose=4,
                    seed= SEED,
                    random_state=1, 
                    silent = False,
                    ),

             XGBClassifier(
                    n_estimators=200,
                    max_depth=10, 
                    verbose = 10, 
                    learning_rate=0.07, 
                    subsample=0.8, 
                    colsample_bytree=0.9,
                    missing=-999,
                    random_state=1, 
                    objective = 'binary:logistic', 
                    tree_method='hist',
                     silent = False),
             
             CatBoostClassifier(
                    n_estimators=200,
                    verbose = 1,
                    max_depth=10, 
                    random_state=1,
                    learning_rate=0.07, 
                    class_weights = [1,28.58],
                     silent = False),
             
               ],
    ######## Second level ########
    
                [    
                    AdaBoostClassifier(
                    algorithm='SAMME.R', 
                    base_estimator=None,
                    learning_rate=0.07, 
                    n_estimators=100, 
                    random_state=1
                    silent = False),

                    GradientBoostingClassifier(
                    n_estimators=100,
                    min_samples_leaf = 100,
                    verbose = 1,
                    min_weight_fraction_leaf=0.0345, 
                    max_depth=10, 
                    max_features=0.7, 
                    learning_rate=0.1, 
                    random_state=1
                    silent = False),
                    ],
                ###this meta model does not need to be deep so n_estimators can be just 200 maybe
    
   ######## Third level ########
           
               [AdaBoostClassifier(
                    algorithm='SAMME.R', 
                    base_estimator=None,
                    learning_rate=0.07, 
                    n_estimators=100, 
                    random_state=1,
                   silent = False
                    ),
            ]
        ]





Fold: [     0      1      2 ... 118105 118106 118107]
Fold: [118108 118109 118110 ... 236213 236214 236215]
Fold: [236216 236217 236218 ... 354321 354322 354323]
Fold: [354324 354325 354326 ... 472429 472430 472431]
Fold: [472432 472433 472434 ... 590537 590538 590539]


In [31]:
import pystacknet
from pystacknet.pystacknet.pystacknet import StackNetClassifier

model=StackNetClassifier(models, metric='auc', folds=5,
   restacking=False,use_retraining=True, use_proba=True, 
   random_state=6,n_jobs=3, verbose=10)


In [32]:
model.fit(X_train.values,y_train.values)

print("Model is fit!!Now predict!!")

====================== Start of Level 0 ======================
Input Dimensionality 315 at Level 0 
3 models included in Level 0 
Fold 1/5 , model 0 , auc===0.936168 
Fold 1/5 , model 1 , auc===0.965853 
Fold 1/5 , model 2 , auc===0.946212 
=========== end of fold 1 in level 0 ===========
Fold 2/5 , model 0 , auc===0.934417 
Fold 2/5 , model 1 , auc===0.964666 
Fold 2/5 , model 2 , auc===0.946771 
=========== end of fold 2 in level 0 ===========
Fold 3/5 , model 0 , auc===0.938391 
Fold 3/5 , model 1 , auc===0.964707 
Fold 3/5 , model 2 , auc===0.949718 
=========== end of fold 3 in level 0 ===========
Fold 4/5 , model 0 , auc===0.933488 
Fold 4/5 , model 1 , auc===0.964958 
Fold 4/5 , model 2 , auc===0.946553 
=========== end of fold 4 in level 0 ===========
Fold 5/5 , model 0 , auc===0.932893 
Fold 5/5 , model 1 , auc===0.964299 
Fold 5/5 , model 2 , auc===0.945938 
=========== end of fold 5 in level 0 ===========
Level 0, model 0 , auc===0.935071 
Level 0, model 1 , auc===0.964897 


c:\users\oem\appdata\local\programs\python\python37\lib\site-packages\joblib\externals\loky\process_executor.py:706: UserWarning:

A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.



Fold 2/5 , model 0 , auc===0.964656 
Fold 2/5 , model 1 , auc===0.964747 
=========== end of fold 2 in level 1 ===========
Fold 3/5 , model 0 , auc===0.964729 
Fold 3/5 , model 1 , auc===0.963098 
=========== end of fold 3 in level 1 ===========


c:\users\oem\appdata\local\programs\python\python37\lib\site-packages\joblib\externals\loky\process_executor.py:706: UserWarning:

A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.



Fold 4/5 , model 0 , auc===0.965067 
Fold 4/5 , model 1 , auc===0.965804 
=========== end of fold 4 in level 1 ===========
Fold 5/5 , model 0 , auc===0.964249 
Fold 5/5 , model 1 , auc===0.963796 
=========== end of fold 5 in level 1 ===========
Level 1, model 0 , auc===0.964925 
Level 1, model 1 , auc===0.964575 
Output dimensionality of level 1 is 2 
====================== End of Level 1 ======================
 level 1 lasted 268.106188 seconds 
====================== Start of Level 2 ======================
Input Dimensionality 2 at Level 2 
1 models included in Level 2 
Fold 1/5 , model 0 , auc===0.967176 
=========== end of fold 1 in level 2 ===========
Fold 2/5 , model 0 , auc===0.966403 
=========== end of fold 2 in level 2 ===========
Fold 3/5 , model 0 , auc===0.965020 
=========== end of fold 3 in level 2 ===========
Fold 4/5 , model 0 , auc===0.967232 
=========== end of fold 4 in level 2 ===========
Fold 5/5 , model 0 , auc===0.965635 
=========== end of fold 5 in level 2 ==

In [33]:

##the test data has to be np array data not dataframe otherwise will through feature name missmatch. 
y_result=model.predict_proba(X_test.values)

====================== Start of Level 0 ======================
1 estimators included in Level 0 
====================== Start of Level 1 ======================
1 estimators included in Level 1 
====================== Start of Level 2 ======================
1 estimators included in Level 2 


In [29]:
##trick here 
##for oof = 0.93547, LB score is 0.9458
##for oof = 0.936, LB score is 0.945

In [37]:

sub = pd.DataFrame(columns=['TransactionID','isFraud'])
sub['TransactionID'] = X_test.index
sub['isFraud'] = y_result[:,1]
sub




sub.to_csv("submission60th.csv",index=False)

sum(y_result[:,1]>0.5)


10509

[[ True False]
 [ True False]
 [ True False]
 ...
 [ True False]
 [ True False]
 [ True False]]
